## Setup

In [1]:
import os
import numpy as np
import time
import cv2
from __future__ import print_function

## Loop

In [95]:
USE_CAM = 1
LOOP = True    # loop through videos
DISP_BW = False

BORDER_STYLE = 0

THRESH = False
THRESH_STYLE = 0
USE_ALL_CHAN = True
USE_R_CHAN = False
USE_G_CHAN = False
USE_B_CHAN = False

GRAD_STYLE = 0

# select video source
if USE_CAM:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture('/media/data/Dropbox/Git/vid-viz/data/test.mp4')
    
# get video properties
im_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
im_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
target_fps = cap.get(cv2.CAP_PROP_FPS)
tot_fr_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fr_count = 0
while(cap.isOpened()):

    time_pre = time.time()
    
    ret, frame = cap.read()

    # use keyboard input to change effect
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        # toggle color
        DISP_BW = not DISP_BW
    elif key == ord('z'):
        BORDER_STYLE = (BORDER_STYLE + 1) % 3
    elif key == ord('t'):
        THRESH = not THRESH
    elif key == ord('y'):
        THRESH_STYLE = (THRESH_STYLE + 1) % 2
    elif key == ord('r'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = True
        USE_G_CHAN = False
        USE_B_CHAN = False
    elif key == ord('g'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = True
        USE_B_CHAN = False
    elif key == ord('b'):
        # for thresholding
        USE_ALL_CHAN = False
        USE_R_CHAN = False
        USE_G_CHAN = False
        USE_B_CHAN = True
    elif key == ord('v'):
        USE_ALL_CHAN = True
        USE_R_CHAN = False
        USE_G_CHAN = False
        USE_B_CHAN = False
    elif key == ord('m'):
        GRAD_STYLE = (GRAD_STYLE + 1) % 2        
        
    # implement effects 
    if DISP_BW:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
#     tx = fr_count % im_height
#     ty = fr_count % im_height
#     M = np.float32([[1, 0, tx], [0, 1, ty]])
#     frame = cv2.warpAffine(frame, M, (im_width, im_height))
    
    if BORDER_STYLE == 0:
        frame = cv2.resize(frame, (1*im_width, 1*im_height), 
                           interpolation=cv2.INTER_LINEAR)
    elif BORDER_STYLE == 1:
        # top, bottom, left, right
        frame = cv2.copyMakeBorder(frame, int(im_height/2), int(im_height/2),
                                          int(im_width/2), int(im_width/2),
                                          cv2.BORDER_WRAP)
    elif BORDER_STYLE == 2:
        # top, bottom, left, right
        frame = cv2.copyMakeBorder(frame, im_height, 0,
                                          im_width, 0,
                                          cv2.BORDER_REFLECT)

        
    if THRESH:
        
        # BINARY | BINARY_INV | TRUNC | TOZERO | TOZERO_INV
        if THRESH_STYLE == 0:
            THRESH_TYPE = cv2.THRESH_BINARY
        elif THRESH_STYLE == 1:
            THRESH_TYPE = cv2.THRESH_BINARY_INV
            
        # MEAN_C | GAUSSIAN_C
        ADAPTIVE_THRESH_TYPE = cv2.ADAPTIVE_THRESH_MEAN_C
        THRESH_CEIL = 255
        THRESH_BLOCK = 21
        THRESH_C = 4
        
        if USE_ALL_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_frame = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL, 
                                          ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                          THRESH_BLOCK, THRESH_C)
            frame[:,:,0] = gray_frame
            frame[:,:,1] = gray_frame
            frame[:,:,2] = gray_frame
            frame = cv2.medianBlur(frame, 11)
        elif USE_R_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out blue channel, threshold red channel
            frame[:,:,0] = 0
            frame[:,:,2] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL, 
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE, 
                                                 THRESH_BLOCK, THRESH_C)
            frame = cv2.medianBlur(frame, 11)
        elif USE_G_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out red channel, threshold green channel
            frame[:,:,2] = 0
            frame[:,:,1] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
            frame = cv2.medianBlur(frame, 11)
        elif USE_B_CHAN:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # zero out green channel, threshold red channel
            frame[:,:,1] = 0
            frame[:,:,0] = cv2.adaptiveThreshold(gray_frame, THRESH_CEIL,
                                                 ADAPTIVE_THRESH_TYPE, THRESH_TYPE,
                                                 THRESH_BLOCK, THRESH_C)
            frame = cv2.medianBlur(frame, 11)
    
#     if GRAD_STYLE == 1:
#         grad = cv2.Sobel(frame, cv2.CV_16S, 1, 0, ksize=7)
#     elif GRAD_STYLE == 2:
#         grad = cv2.Sobel(frame, cv2.CV_16S, 0, 1, ksize=7)
#     elif GRAD_STYLE == 3:
#         grad = cv2.Scharr(frame, cv2.CV_16S, 1, 0)
#     elif GRAD_STYLE == 4:
#         grad = cv2.Scharr(frame, cv2.CV_16S, 0, 1)
#     elif GRAD_STYLE == 5:
#         grad = cv2.Laplacian(frame, cv2.CV_16S, ksize=7)
#     elif GRAD_STYLE == 6:
#         grad = cv2.Canny(frame, 100, 200)   

    if GRAD_STYLE > 0:
        grad = cv2.Canny(frame, 100, 200)  
        if USE_R_CHAN:
            frame[:,:,:2] = 0
            frame[:,:,2] = grad
        elif USE_G_CHAN:
            frame[:,:,[0,2]] = 0
            frame[:,:,1] = grad
        elif USE_B_CHAN:
            frame[:,:,1:3] = 0
            frame[:,:,0] = grad
        elif USE_ALL_CHAN:
            frame = grad
    
    # display image
    cv2.imshow('frame', frame)
    
    # reset video
    fr_count += 1
    if fr_count == tot_fr_count and LOOP:
        # reset frame postion to zero
        cap.set(2,0)
        fr_count = 0
    
    # calculate, limit and output fps
    time_post = time.time()
    time_tot = time_post - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_post = time.time()
    time_tot = time_post - time_pre
    fps = 1.0 / time_tot
    print('\r%03i fps' % fps, end='')
    
    
cap.release()
cv2.destroyWindow('frame')

014 fps

In [28]:
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(flags[:2])

['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGR_EA']


In [2]:
img = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test.jpg', 0) 
img.dtype

dtype('uint8')

In [2]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'dv2' is not defined

In [20]:
whos

Variable               Type                   Data/Info
-------------------------------------------------------
ADAPTIVE_THRESH_TYPE   int                    0
BORDER_STYLE           int                    0
DISP_BORDER            int                    1
DISP_BW                bool                   False
DISP_B_CHAN            bool                   True
DISP_G_CHAN            bool                   True
DISP_R_CHAN            bool                   True
LOOP                   bool                   True
M                      ndarray                2x3: 6 elems, type `float32`, 24 bytes
MORPHO_STYLE           int                    0
THRESH                 bool                   False
THRESH_BLOCK           int                    21
THRESH_C               int                    4
THRESH_CEIL            int                    255
THRESH_STYLE           int                    0
THRESH_TYPE            int                    0
USE_ALL_CHAN           bool                   True
USE_B_CHA